In [1]:
import os
os.chdir("../")
%pwd

'/home/dheeraj37/Projects/Audio-Genre-Classification-1'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    model_path: Path
    trained_model_path: Path
    train_data_path: Path
    val_data_path: Path
    test_data_path: Path
    params_epochs: int
    params_batch_size: int
    params_shuffle: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path



In [3]:
from cnnClassifier.constant import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


2023-09-23 23:18:12.699692: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-23 23:18:12.700841: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 23:18:12.723269: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 23:18:12.723836: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 23:18:13.144915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )

        return prepare_callback_config


    def get_training_config(self) -> TrainingConfig:
        model_training = self.config.model_training
        prepare_base_model = self.config.prepare_base_model
        prepare_data = self.config.prepare_data
        params = self.params
        create_directories([
            Path(model_training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(model_training.root_dir),
            model_path=Path(prepare_base_model.model_path),
            trained_model_path=Path(model_training.trained_model_path),
            train_data_path=Path(prepare_data.train_data_path), 
            test_data_path=Path(prepare_data.test_data_path),
            val_data_path=Path(prepare_data.val_data_path),     
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_shuffle=params.SHUFFLE,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config



In [5]:
import time

In [6]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from cnnClassifier.utils.common import MFCCDataGenerator

from sklearn.model_selection import train_test_split

In [15]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        
    def get_model(self):
        self.model = tf.keras.models.load_model(
            self.config.model_path
        )

    def get_datasets(self):
        self.train_df = pd.read_json(self.config.train_data_path)
        self.val_df = pd.read_json(self.config.val_data_path)
        self.test_df = pd.read_json(self.config.test_data_path)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.train_data_generator = MFCCDataGenerator(self.train_df, batch_size=64, shuffle=True)
        self.val_data_generator = MFCCDataGenerator(self.val_df, batch_size=64, shuffle=False)

        #self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        #self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.history = self.model.fit(
            self.train_data_generator,
            epochs=self.config.params_epochs,
            #steps_per_epoch=self.steps_per_epoch,
            #validation_steps=self.validation_steps,
            validation_data=self.val_data_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [20]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_model()
    training.get_datasets()
    #training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-09-23 23:26:11,607]: INFO: common: yaml file: config/config.yaml loaded successfully
[2023-09-23 23:26:11,608]: INFO: common: yaml file: params.yaml loaded successfully
[2023-09-23 23:26:11,608]: INFO: common: created directory at: artifacts
[2023-09-23 23:26:11,608]: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir
[2023-09-23 23:26:11,609]: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
[2023-09-23 23:26:11,609]: INFO: common: created directory at: artifacts/model_training
Epoch 1/100
64/64 [==============================] - 2s 24ms/step - loss: 3.2780 - accuracy: 0.1573 - val_loss: 2.4608 - val_accuracy: 0.1335
Epoch 2/100
 7/64 [==>...........................] - ETA: 0s - loss: 2.6830 - accuracy: 0.2098

/home/dheeraj37/miniconda3/envs/agc/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


64/64 [==============================] - 1s 23ms/step - loss: 2.5056 - accuracy: 0.2145 - val_loss: 1.9917 - val_accuracy: 0.2993
Epoch 3/100
 7/64 [==>...........................] - ETA: 1s - loss: 2.3528 - accuracy: 0.2482

/home/dheeraj37/miniconda3/envs/agc/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


64/64 [==============================] - 1s 22ms/step - loss: 2.2159 - accuracy: 0.2582 - val_loss: 1.7851 - val_accuracy: 0.3739
Epoch 4/100
 8/64 [==>...........................] - ETA: 1s - loss: 2.1034 - accuracy: 0.2773

...

/home/dheeraj37/miniconda3/envs/agc/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


64/64 [==============================] - 2s 25ms/step - loss: 0.9447 - accuracy: 0.6466 - val_loss: 1.0066 - val_accuracy: 0.6516
Epoch 73/100
 7/64 [==>...........................] - ETA: 1s - loss: 0.9582 - accuracy: 0.6205

/home/dheeraj37/miniconda3/envs/agc/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


64/64 [==============================] - 2s 26ms/step - loss: 0.9347 - accuracy: 0.6471 - val_loss: 1.0370 - val_accuracy: 0.6359
Epoch 74/100
64/64 [==============================] - 2s 26ms/step - loss: 0.9398 - accuracy: 0.6491 - val_loss: 1.0091 - val_accuracy: 0.6546
Epoch 75/100
64/64 [==============================] - 2s 26ms/step - loss: 0.9100 - accuracy: 0.6648 - val_loss: 1.0421 - val_accuracy: 0.6251
Epoch 76/100
64/64 [==============================] - 2s 25ms/step - loss: 0.9186 - accuracy: 0.6596 - val_loss: 1.0150 - val_accuracy: 0.6497
Epoch 77/100
64/64 [==============================] - 2s 26ms/step - loss: 0.8956 - accuracy: 0.6712 - val_loss: 1.0278 - val_accuracy: 0.6467
Epoch 78/100
64/64 [==============================] - 2s 25ms/step - loss: 0.8995 - accuracy: 0.6623 - val_loss: 1.0217 - val_accuracy: 0.6506
Epoch 79/100
64/64 [==============================] - 2s 25ms/step - loss: 0.8943 - accuracy: 0.6734 - val_loss: 1.0260 - val_accuracy: 0.6506
Epoch 80/100

/home/dheeraj37/miniconda3/envs/agc/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def train_valid_generator(self): 

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )